# Chapter 20: Integration and Final Projects

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adiel2012/computer-vision/blob/main/chapter_20_integration_and_projects.ipynb)

**Integration and projects** bring together all techniques from the previous chapters into complete rendering systems. This final chapter guides you through building production-quality renderers and provides project ideas for further exploration.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
from typing import Tuple, List, Optional

## 1. System Architecture

Building a complete rendering system requires careful integration of all components.

### Renderer Architecture Patterns

**1. Scene Graph**
```
Scene
├── Camera
├── Lights[]
├── Materials[]
├── Meshes[]
│   ├── Geometry
│   ├── Material
│   └── Transform
└── Acceleration Structure (BVH)
```

**2. Render Pipeline**
```
Input (Scene + Camera)
   ↓
Geometry Processing
   ↓
Acceleration Structure Build
   ↓
Rendering (Rasterize or Ray Trace)
   ↓
Post-Processing
   ↓
Output (Image)
```

**3. Component Relationships**

- **Scene** owns all objects and manages lifetime
- **Camera** defines view transformation
- **Materials** describe surface properties (BRDF)
- **Lights** provide illumination
- **Acceleration structures** enable fast ray queries
- **Framebuffer** stores rendered output

### Design Principles

✅ **Modularity**: Separate concerns (geometry, shading, acceleration)
✅ **Extensibility**: Easy to add new primitives, materials, lights
✅ **Performance**: Cache-friendly data structures
✅ **Testability**: Unit test individual components
✅ **Configurability**: JSON/YAML scene files

In [ ]:
from dataclasses import dataclass, field
from enum import Enum
from typing import Dict, Any
import json

# ============================================================================
# COMPLETE INTEGRATED RENDERER SYSTEM
# ============================================================================

# Vector and Matrix classes (simplified versions from Chapter 1)
class Vec3:
    """3D Vector"""
    def __init__(self, x=0.0, y=0.0, z=0.0):
        self.x = x
        self.y = y
        self.z = z
    
    def __add__(self, other):
        return Vec3(self.x + other.x, self.y + other.y, self.z + other.z)
    
    def __sub__(self, other):
        return Vec3(self.x - other.x, self.y - other.y, self.z - other.z)
    
    def __mul__(self, scalar):
        return Vec3(self.x * scalar, self.y * scalar, self.z * scalar)
    
    def dot(self, other):
        return self.x * other.x + self.y * other.y + self.z * other.z
    
    def cross(self, other):
        return Vec3(
            self.y * other.z - self.z * other.y,
            self.z * other.x - self.x * other.z,
            self.x * other.y - self.y * other.x
        )
    
    def length(self):
        return math.sqrt(self.dot(self))
    
    def normalize(self):
        l = self.length()
        if l > 0:
            return Vec3(self.x / l, self.y / l, self.z / l)
        return Vec3(0, 0, 0)
    
    def to_array(self):
        return np.array([self.x, self.y, self.z])

@dataclass
class Ray:
    """Ray with origin and direction"""
    origin: Vec3
    direction: Vec3
    
    def at(self, t: float) -> Vec3:
        """Get point at parameter t"""
        return self.origin + self.direction * t

@dataclass
class HitRecord:
    """Intersection record"""
    t: float = float('inf')
    point: Vec3 = field(default_factory=Vec3)
    normal: Vec3 = field(default_factory=Vec3)
    material_id: int = -1
    hit: bool = False

class MaterialType(Enum):
    """Material types"""
    LAMBERTIAN = 1
    METAL = 2
    DIELECTRIC = 3
    PBR = 4

@dataclass
class Material:
    """Material properties"""
    type: MaterialType
    albedo: Vec3 = field(default_factory=lambda: Vec3(0.8, 0.8, 0.8))
    roughness: float = 0.5
    metallic: float = 0.0
    ior: float = 1.5
    emission: Vec3 = field(default_factory=Vec3)

class Sphere:
    """Sphere primitive"""
    def __init__(self, center: Vec3, radius: float, material_id: int):
        self.center = center
        self.radius = radius
        self.material_id = material_id
    
    def intersect(self, ray: Ray) -> HitRecord:
        """Ray-sphere intersection"""
        oc = ray.origin - self.center
        a = ray.direction.dot(ray.direction)
        half_b = oc.dot(ray.direction)
        c = oc.dot(oc) - self.radius * self.radius
        
        discriminant = half_b * half_b - a * c
        
        hit = HitRecord()
        
        if discriminant < 0:
            return hit
        
        sqrtd = math.sqrt(discriminant)
        root = (-half_b - sqrtd) / a
        
        if root < 0.001 or root > 1000:
            root = (-half_b + sqrtd) / a
            if root < 0.001 or root > 1000:
                return hit
        
        hit.t = root
        hit.point = ray.at(root)
        hit.normal = (hit.point - self.center) * (1.0 / self.radius)
        hit.material_id = self.material_id
        hit.hit = True
        
        return hit

@dataclass
class Light:
    """Light source"""
    position: Vec3
    color: Vec3
    intensity: float = 1.0

class Camera:
    """Pinhole camera"""
    def __init__(self, look_from: Vec3, look_at: Vec3, vup: Vec3, vfov: float, aspect: float):
        theta = vfov * math.pi / 180.0
        h = math.tan(theta / 2.0)
        viewport_height = 2.0 * h
        viewport_width = aspect * viewport_height
        
        w = (look_from - look_at).normalize()
        u = vup.cross(w).normalize()
        v = w.cross(u)
        
        self.origin = look_from
        self.horizontal = u * viewport_width
        self.vertical = v * viewport_height
        self.lower_left = self.origin - self.horizontal * 0.5 - self.vertical * 0.5 - w
    
    def get_ray(self, u: float, v: float) -> Ray:
        """Get ray through (u, v) on viewport"""
        direction = (self.lower_left + self.horizontal * u + self.vertical * v - self.origin).normalize()
        return Ray(self.origin, direction)

class Scene:
    """Complete scene representation"""
    def __init__(self):
        self.objects: List[Sphere] = []
        self.materials: List[Material] = []
        self.lights: List[Light] = []
        self.camera: Optional[Camera] = None
        self.background = Vec3(0.5, 0.7, 1.0)  # Sky color
    
    def add_sphere(self, center: Vec3, radius: float, material: Material) -> int:
        """Add sphere to scene"""
        mat_id = len(self.materials)
        self.materials.append(material)
        self.objects.append(Sphere(center, radius, mat_id))
        return len(self.objects) - 1
    
    def add_light(self, position: Vec3, color: Vec3, intensity: float = 1.0):
        """Add light to scene"""
        self.lights.append(Light(position, color, intensity))
    
    def set_camera(self, camera: Camera):
        """Set scene camera"""
        self.camera = camera
    
    def intersect(self, ray: Ray) -> HitRecord:
        """Find closest intersection"""
        closest = HitRecord()
        
        for obj in self.objects:
            hit = obj.intersect(ray)
            if hit.hit and hit.t < closest.t:
                closest = hit
        
        return closest

print("✓ Core rendering system classes loaded")

In [ ]:
def create_demo_scene() -> Scene:
    """Create a demonstration scene with various materials"""
    scene = Scene()
    
    # Ground (large sphere)
    ground_mat = Material(type=MaterialType.LAMBERTIAN, albedo=Vec3(0.5, 0.5, 0.5))
    scene.add_sphere(Vec3(0, -1000, 0), 1000, ground_mat)
    
    # Center sphere (diffuse red)
    center_mat = Material(type=MaterialType.LAMBERTIAN, albedo=Vec3(0.7, 0.3, 0.3))
    scene.add_sphere(Vec3(0, 1, 0), 1.0, center_mat)
    
    # Left sphere (metal)
    left_mat = Material(type=MaterialType.METAL, albedo=Vec3(0.8, 0.8, 0.8))
    scene.add_sphere(Vec3(-2.5, 1, 0), 1.0, left_mat)
    
    # Right sphere (gold metal)
    right_mat = Material(type=MaterialType.METAL, albedo=Vec3(0.8, 0.6, 0.2))
    scene.add_sphere(Vec3(2.5, 1, 0), 1.0, right_mat)
    
    # Small spheres
    small_mat1 = Material(type=MaterialType.LAMBERTIAN, albedo=Vec3(0.1, 0.5, 0.8))
    scene.add_sphere(Vec3(-1, 0.5, 2), 0.5, small_mat1)
    
    small_mat2 = Material(type=MaterialType.LAMBERTIAN, albedo=Vec3(0.9, 0.9, 0.1))
    scene.add_sphere(Vec3(1, 0.5, 2), 0.5, small_mat2)
    
    # Lights
    scene.add_light(Vec3(-5, 10, -5), Vec3(1, 1, 1), 0.8)
    scene.add_light(Vec3(5, 5, 5), Vec3(1, 0.9, 0.8), 0.5)
    
    # Camera
    camera = Camera(Vec3(8, 3, 8), Vec3(0, 1, 0), Vec3(0, 1, 0), 40.0, 16.0/9.0)
    scene.set_camera(camera)
    
    return scene

# Create and render
scene = create_demo_scene()
renderer = IntegratedRenderer(400, 225)
image = renderer.render(scene, samples_per_pixel=4)

# Display
plt.figure(figsize=(14, 8))
plt.imshow(image)
plt.title('Complete Integrated Renderer\n(Ray Tracing + Shadows + Reflection + Multiple Materials)', 
          fontsize=14, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("RENDERING FEATURES DEMONSTRATED:")
print("="*70)
print("✓ Ray-sphere intersection")
print("✓ Recursive ray tracing (reflection)")
print("✓ Shadow rays (hard shadows)")
print("✓ Multiple material types (Lambertian, Metal)")
print("✓ Multiple light sources")
print("✓ Phong-like shading model")
print("✓ Anti-aliasing (4 SPP with jittering)")
print("✓ Gamma correction")
print("✓ Sky background gradient")
print("="*70)

## 3. Scene Setup and Rendering

Let's create a complete demo scene and render it!

In [ ]:
import random

class IntegratedRenderer:
    """Complete renderer with all features"""
    
    def __init__(self, width: int, height: int):
        self.width = width
        self.height = height
        self.framebuffer = np.zeros((height, width, 3))
        self.max_depth = 5
    
    def reflect(self, v: Vec3, n: Vec3) -> Vec3:
        """Reflect vector v about normal n"""
        return v - n * (2.0 * v.dot(n))
    
    def shade_lambertian(self, hit: HitRecord, scene: Scene, material: Material) -> Vec3:
        """Lambertian (diffuse) shading"""
        color = Vec3(0, 0, 0)
        
        for light in scene.lights:
            # Direction to light
            light_dir = (light.position - hit.point).normalize()
            
            # Shadow ray
            shadow_ray = Ray(hit.point, light_dir)
            shadow_hit = scene.intersect(shadow_ray)
            
            # Check if in shadow
            dist_to_light = (light.position - hit.point).length()
            if shadow_hit.hit and shadow_hit.t < dist_to_light:
                continue  # In shadow
            
            # Diffuse term
            diffuse = max(0.0, hit.normal.dot(light_dir))
            
            # Accumulate light contribution
            color = color + Vec3(
                material.albedo.x * light.color.x * diffuse * light.intensity,
                material.albedo.y * light.color.y * diffuse * light.intensity,
                material.albedo.z * light.color.z * diffuse * light.intensity
            )
        
        # Ambient
        ambient = Vec3(0.1, 0.1, 0.1)
        color = color + Vec3(
            material.albedo.x * ambient.x,
            material.albedo.y * ambient.y,
            material.albedo.z * ambient.z
        )
        
        return color
    
    def shade_metal(self, ray: Ray, hit: HitRecord, scene: Scene, material: Material, depth: int) -> Vec3:
        """Metal (reflective) shading"""
        reflected = self.reflect(ray.direction, hit.normal)
        reflected_ray = Ray(hit.point, reflected)
        
        # Recursive reflection
        reflected_color = self.trace_ray(reflected_ray, scene, depth + 1)
        
        return Vec3(
            material.albedo.x * reflected_color.x,
            material.albedo.y * reflected_color.y,
            material.albedo.z * reflected_color.z
        )
    
    def trace_ray(self, ray: Ray, scene: Scene, depth: int = 0) -> Vec3:
        """Trace a ray through the scene"""
        
        # Max recursion depth
        if depth >= self.max_depth:
            return Vec3(0, 0, 0)
        
        # Intersect scene
        hit = scene.intersect(ray)
        
        if not hit.hit:
            # Background color (sky)
            t = 0.5 * (ray.direction.y + 1.0)
            return Vec3(
                (1.0 - t) * 1.0 + t * scene.background.x,
                (1.0 - t) * 1.0 + t * scene.background.y,
                (1.0 - t) * 1.0 + t * scene.background.z
            )
        
        # Get material
        material = scene.materials[hit.material_id]
        
        # Emission (for lights)
        color = material.emission
        
        # Shade based on material type
        if material.type == MaterialType.LAMBERTIAN:
            color = color + self.shade_lambertian(hit, scene, material)
        elif material.type == MaterialType.METAL:
            color = color + self.shade_metal(ray, hit, scene, material, depth)
        
        return color
    
    def render(self, scene: Scene, samples_per_pixel: int = 1) -> np.ndarray:
        """Render the scene"""
        if scene.camera is None:
            raise ValueError("Scene must have a camera")
        
        print(f"Rendering {self.width}x{self.height} at {samples_per_pixel} SPP...")
        
        for y in range(self.height):
            if y % 20 == 0:
                print(f"  Scanline {y}/{self.height}")
            
            for x in range(self.width):
                color = Vec3(0, 0, 0)
                
                # Multi-sampling
                for _ in range(samples_per_pixel):
                    # Normalized coordinates with jitter
                    u = (x + random.random()) / (self.width - 1)
                    v = 1.0 - (y + random.random()) / (self.height - 1)
                    
                    ray = scene.camera.get_ray(u, v)
                    sample_color = self.trace_ray(ray, scene)
                    color = color + sample_color
                
                # Average samples
                color = color * (1.0 / samples_per_pixel)
                
                # Gamma correction
                color = Vec3(
                    math.sqrt(max(0, color.x)),
                    math.sqrt(max(0, color.y)),
                    math.sqrt(max(0, color.z))
                )
                
                # Clamp and store
                self.framebuffer[y, x] = [
                    min(1.0, color.x),
                    min(1.0, color.y),
                    min(1.0, color.z)
                ]
        
        print("✓ Rendering complete!")
        return self.framebuffer

print("✓ Integrated renderer loaded")

## 2. Complete Renderer Implementation

Now let's build a complete renderer that integrates all techniques:
- Ray tracing with reflection
- Phong shading
- Shadows
- Multiple materials

## 4. Render the Scene

Now let's render our complete integrated scene!

## 2. Complete Renderer Implementation

Now let's build a complete renderer that integrates all techniques:
- Ray tracing with reflection
- Phong shading
- Shadows
- Multiple materials